# *KAGGLE CHALLENGE: LANL Earthquake Prediction*

Un projet de Matthieu Dagommer, Paul Boulgakoff, Godefroy Bichon, Germain L'Hostis

Versions utilisées:

Python: 3.10.4
Torch: 1.11

In [1]:
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt 

import torch as th
th.cuda.empty_cache()

import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
#from torchviz import make_dot

from torchsummary import summary 

import gzip
import pickle

import pandas as pd
import scipy
import scipy.stats as stats
from scipy.stats import kurtosis, skew
import csv
import os
import pickle
import gc

from sklearn.preprocessing import StandardScaler, MinMaxScaler

seed = 1
th.manual_seed(seed)
np.random.seed(seed)

In [2]:
### Ensuring CPU is connected

print(th.cuda.is_available())
print(th.cuda.get_device_name())
device = th.device('cuda' if th.cuda.is_available() else 'cpu')

True
GeForce GTX 1050 Ti with Max-Q Design


# *General Hyperparameters*

In [3]:
### Setting General Hyperparameters 

size_batch = 100 # nombres de patchs par batchs
valid_rate = 0.1 # proportion des données disponibles consacrée à la validation
overlap_rate = 0.2 # taux de recouvrement 

# *Data Preparation*



In [4]:
### Loading Data

rootpath = os.getcwd() + "/"
nrows = 200000000

# Possible de tout charger dans la RAM comme ceci (prend 9 GB environ si on charge les 629 145 480 lignes)
train_data = pd.read_csv(rootpath + "train.csv", usecols = ['acoustic_data', 'time_to_failure'], \
                         dtype = {'acoustic_data': np.int16, 'time_to_failure': np.float64}, nrows = nrows)

In [5]:
X = th.squeeze(th.tensor(train_data['acoustic_data'].values, dtype = th.int16))
Y = th.squeeze(th.tensor(train_data['time_to_failure'].values, dtype = th.float64))

del train_data
gc.collect()

0

In [6]:
### Retrieve Test Data

path = rootpath + "test/" + "seg_00030f.csv"
test = pd.read_csv(path)
time_window = test.shape[0]

# Les lignes ci-dessous prennent du temps à calculer et servent à charger les données test,
# dont on a pas besoin pour le moment.
# elles ont été enlevées temporairement pour lancer le code sous drive:

#path = rootpath + "sample_submission.csv"
#sample_submission = pd.read_csv(path)


#checking the ttf of test samples (ALL ZEROS ?!?)
#test_ttf_counts = sample_submission.time_to_failure.value_counts()


#Importation des données à prédire
#X_test = []

#for filename in os.listdir(rootpath + "test"):
#    temp_df = pd.read_csv(rootpath + "test/" + filename)
#    X_test.append(temp_df)

#Xtest = th.zeros((len(X_test),time_window))
#for i in range(len(X_test)):
#  Xtest[i,:] = th.tensor(X_test[i]["acoustic_data"], dtype = th.float32)

#Ytest = th.tensor(sample_submission["time_to_failure"])

In [7]:
size_patch = time_window # taille des patchs calquée sur la longueur des séries test

In [8]:
### Function to create patched sequences with some overlap out of the training data

# Le recouvrement est exprimé en proportion de la taille d'une fenêtre (time_window)
def patching(size_patch, X, Y, overlap_rate):
    
    overlap = int(size_patch*overlap_rate)
    L = X.shape[0] # total length of the training acoustic signal
    n_patch = int(np.floor((L-size_patch)/(size_patch-overlap)+1))
    ids_no_seism = []

    X_patch = th.zeros(n_patch,size_patch)
    Y_patch = th.zeros(n_patch)
    
    for i in range (n_patch):
        X_patch[i,:] = X[i*(size_patch-overlap):(i+1)*size_patch-i*overlap] 
        Y_patch[i] = Y[(i+1)*size_patch - i*overlap] 
    
        # Relever les patchs qui ne contiennent pas de séisme
        if th.min(Y[i*(size_patch-overlap):(i+1)*size_patch - i*overlap]) > 0.001:
            ids_no_seism.append(i)
    
    X_patch = X_patch[ids_no_seism]
    Y_patch = Y_patch[ids_no_seism]
    
    return(n_patch, X_patch, Y_patch)

In [9]:
### Normalizing Data

ss = StandardScaler()
mm = MinMaxScaler()
    
X = th.squeeze(th.from_numpy(ss.fit_transform(np.array(X).reshape(-1, 1))))
Y = th.squeeze(th.from_numpy(mm.fit_transform(np.array(Y).reshape(-1, 1))))

In [10]:
### Distribution of data in patches

_, X_patch, Y_patch = patching(size_patch, X, Y, overlap_rate = overlap_rate)

del X; del Y
gc.collect()

print(X_patch.shape)
print("There are ", X_patch.shape[0], "time series available for training and validation after patching with overlap. \n")

torch.Size([1656, 150000])
There are  1656 time series available for training and validation after patching with overlap. 



In [11]:
### Initializating Data Sets


# shuffling data
idx = np.arange(X_patch.shape[0]) # indices
shuffled_idx = np.random.shuffle(idx)
X_patch = th.squeeze(X_patch[shuffled_idx,:])
Y_patch = th.squeeze(Y_patch[shuffled_idx])

N_samples = X_patch.shape[0]
N_valid = int(valid_rate*N_samples) # number of validation patches

X_valid = X_patch[:N_valid,:].cpu()
Y_valid = Y_patch[:N_valid].cpu()

# Les inputs de nn.Conv1d doivent être de la forme (N, C_in, *),
#avec N: nombre de samples/taille batch, C_in: le nombre de channels et * les dimensions des objets (nos séries)
# on rajoute la dimension channel (1):
X_valid = th.unsqueeze(X_valid, 1)

#On fait la même chose pour les données X_train, mais directement au moment de constituer les batchs dans la for-loop !
X_train = X_patch[N_valid:,:].cpu()
Y_train = Y_patch[N_valid:].cpu()
N_train = X_train.shape[0]

n_batch = int(X_train.shape[0] / size_batch) # nombre total de batchs

X_train = X_train.reshape([N_train, 1, size_patch])

# *Function for Results plotting*

In [12]:
### Plotting Graphs

def plot_and_save_results(train_losses, valid_losses, best_mvd, best_mtd, mm, model_name):

    best_epoch = np.fromiter(valid_losses, dtype=float).argmin()
    
    fig, ax = plt.subplots(1, 2, figsize = (20,10))
    plt.rcParams['font.size'] = '20'
    ax[0].set(title = "Losses: Training and Validation") 
    ax[0].set_xlabel("epochs", fontsize = 20)
    ax[0].set_ylabel("MSE", fontsize = 20)
    ax[0].plot(train_losses,"r", label = "Training", linewidth = 3)
    ax[0].plot(valid_losses, "b", label = "Validation", linewidth = 3)
    ax[0].legend(loc = "upper right", fontsize = 18)
    ax[0].axvline(x=int(best_epoch), color = 'black', linestyle ="--", linewidth = 3)
    ax[0].annotate("Best epoch: {}\nMSE_train: {:.3f}\nMSE_valid: {:.3f}".format(int(best_epoch), min_tl, min_vl), \
                   xy = (0.5,0.5), xycoords = 'axes fraction')
    
    best_mvd_plot = mm.inverse_transform(best_mvd.reshape(-1, 1))
    best_mtd_plot = mm.inverse_transform(best_mtd.reshape(-1, 1))

    N_valid = best_mvd_plot.shape[0]
    
    mean = float(np.mean(best_mvd_plot))
    std_dev = float(np.std(best_mvd_plot))
    kurt = float(kurtosis(best_mvd_plot))
    skewn = float(skew(best_mvd_plot))
    q1 = float(np.quantile(best_mvd_plot, 0.25))
    median = float(np.quantile(best_mvd_plot, 0.5))
    q3 = float(np.quantile(best_mvd_plot, 0.75))
    mae = np.absolute(best_mvd_plot).sum() / N_valid
    mse = np.square(best_mvd_plot).sum() / N_valid
    
    text = "mean: {:.3f}\nstd: {:.3f}\nkurt: {:.3f}\nskew: {:.3f}\nq1: {:.3f}\nmed: {:.3f}\nq3: {:.3f}\niqr: {:.3f}\nmae: {:.3f}\nmse: {:.3f}\n".format(mean, std_dev, kurt, skewn, q1, median, q3, q3-q1, mae, mse)
    
    ax[1].hist(best_mvd_plot, alpha = 0.3, label = "validation set", bins = 100, density = True, range = (-16, 16))
    ax[1].set(title = "TTF error distributions at best epoch")
    ax[1].set_xlabel("Error (seconds)", fontsize = 20)
    ax[1].set_ylabel("Density", fontsize = 20)
    ax[1].hist(best_mtd_plot, alpha = 0.3, label = "training set", bins = 100, density = True, range = (-16, 16))
    ax[1].annotate(text, xy =(-15, 0.1))
    ax[1].legend()


    plt.gcf()
    plt.savefig(model_name + "_plot.jpg")
    plt.show()

    model_features = {"N_samples": N_samples, "N_train": N_train, "N_valid": N_valid, \
                      "overlap_rate": overlap_rate, "learning_rate": learning_rate, "num_epochs": num_epochs, \
                     "seed": seed, "size_batch": size_batch, "train_losses": train_losses, "valid_losses": valid_losses, \
                     "valid_differences": valid_differences, "best_mtd": best_mtd, "best_mvd": best_mvd, "min_tl": min_tl, \
                      "min_vl": min_vl}

    pickle.dump(model_features, open(model_name + ".p", "wb" ))
    
    model_features_display = {"N_samples": N_samples, "N_train": N_train, "N_valid": N_valid, \
                  "overlap_rate": overlap_rate, "learning_rate": learning_rate, "num_epochs": num_epochs, \
                 "seed": seed, "size_batch": size_batch}
    
    pickle.dump(model_features_display, open(model_name + "display.p", "wb" ))

# *1D Convolutional Neural Network*

Cette partie contient les architectures de deux modèles de convolution 1D qui ont été testés au cours du projet, ainsi que la définition des hyperparamètres et la boucle d'entraînement associée.

Pour lancer les calculs avec le LSTM, vous pouvez sauter ces cellules et vous rendre à la partie concernée située plus bas !

In [ ]:
### Hyperparameters specific to 1D CNN

learning_rate=0.0001
num_epochs=500


In [ ]:
### 1D Convolutional Network #1

class NN(nn.Module):
    
    def __init__ (self,num_classes=1):
        super(NN, self).__init__()
        self.fc1=nn.Conv1d(in_channels=1, out_channels=1, 
                             kernel_size=10, padding=1, stride=5)
        self.fc2=nn.AdaptiveMaxPool1d(200)
        self.fct3=nn.Linear(200,50)
        self.fct4=nn.Linear(50,num_classes)
    
    def forward (self,x):
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fct3(x)
        x=self.fct4(x)
        return x
    

In [ ]:
### 1D Convolutional Network #2

class NN_2(nn.Module):
    def __init__(self,num_classes=1):
        super(NN_2, self).__init__()
        self.seq_1=nn.Sequential(nn.Conv1d(in_channels = 1, out_channels = 16, kernel_size = 10, stride = 5),
                                 nn.ReLU(),
                                 nn.MaxPool1d(kernel_size = 2),
                                 nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 10, stride = 5, \
                                           padding = 'valid'),
                                 nn.ReLU(),
                                 nn.MaxPool1d(kernel_size = 2),
                                 nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 10, stride = 5, \
                                           padding = 'valid'),
                                 nn.ReLU()
                                )
        
        self.seq_2=nn.Sequential(nn.Dropout(p = 0.4),
                                 nn.Linear(in_features = 64, out_features = 1)
                                )
    def forward(self,x):
        x=self.seq_1(x)
        x = th.mean(x, dim = 2, keepdim = True)
        x = th.squeeze(x)
        x=self.seq_2(x)
        return (x)

In [ ]:
### Initialize Network, Define Loss and optimizer

model = NN_2()
print(model)
model.to(device)
#summary(model, (1, time_window))
loss=nn.MSELoss()
optimizer=th.optim.Adam(model.parameters(),learning_rate)

#yhat = model(X_valid.to(device))
#make_dot(yhat, params=dict(list(model.named_parameters()))).render("cnn_2", format="png")

In [ ]:
### Check accuracy during training

def check_accuracy(X_valid, Y_valid, model):
    
    device = 'cuda'
    N_valid = X_valid.shape[0]
    real_ttf = th.squeeze(Y_valid)
    loss_fn = nn.MSELoss()
    
    with th.no_grad():

        scores = th.squeeze(model(X_valid.to(device))).cpu()
        valid_loss_ = loss_fn(scores, real_ttf).cpu()
        valid_differences = scores[:] - real_ttf[:]
        valid_differences = valid_differences.cpu().numpy()
        valid_loss = valid_loss_.item()
    
    return valid_loss, valid_differences

In [ ]:
### Train network

import time
th.cuda.empty_cache()

train_losses = []
valid_losses = []

best_mvd = [] # valid differences at best epoch
best_mtd = [] # training differences at best epoch
min_vl = 1000

N_train_per_epoch = n_batch*size_batch 

start = time.perf_counter()

for epoch in range(num_epochs):

    th.cuda.empty_cache()
    
    # A chaque époque, on constitue aléatoirement de nouveaux batchs
    random_idx = np.random.randint(0, N_train, N_train_per_epoch)
    X_train_batch = th.reshape(X_train[random_idx], [n_batch, size_batch, 1, size_patch])
    Y_train_batch = th.reshape(Y_train[random_idx], [n_batch, size_batch, 1, 1])
    
    running_loss = 0
    _loss = 0

    for ids in range (n_batch):
        
        #On met les gradients à zéro 
        optimizer.zero_grad()

        # Prédiction du modèle
        predicted_ttfs = th.squeeze(model(X_train_batch[ids].to(device))).cpu()
        # Calcul de la fonction de perte
        _loss = loss(predicted_ttfs, th.squeeze(Y_train_batch[ids]))
        
        del predicted_ttfs
        gc.collect()
        
        # Backpropagation
        _loss.backward()
        running_loss += _loss.item()
        
        # Mise à jour des paramètres du modèle
        optimizer.step()
        
    optimizer.zero_grad()
    th.cuda.empty_cache()
    
    valid_loss, valid_differences = check_accuracy(X_valid, Y_valid, model)
    valid_losses.append(valid_loss)
    train_losses.append(running_loss / n_batch)
    
    if valid_loss < min_vl:
        
        min_vl = valid_loss
        min_tl = running_loss
        
        best_mvd = valid_differences
        
        Y_final = th.squeeze(model(X_train[:int(N_train/10)].to(device))).cpu()
        
        best_mtd = Y_train[:int(N_train/10)] - Y_final[:]
        best_mtd = best_mtd.cpu().detach().numpy()
        
    
    if epoch%10 == 0:
        print("Epoch: {}\t".format(epoch),
                "train Loss: {:.5f}.. ".format(train_losses[-1]),
                "valid Loss: {:.5f}.. ".format(valid_losses[-1])) 

print("---------- Best : {:.3f}".format(min(valid_losses)), " at epoch " 
    , np.fromiter(valid_losses, dtype=float).argmin(), " / ", epoch + 1)


end = time.perf_counter()
print("\ntime elapsed: {:.3f}".format(end - start))

In [ ]:
model_name = input("Choose a name for the model: ")

In [ ]:
plot_and_save_results(train_losses, valid_losses, best_mvd, best_mtd, mm, model_name)

In [ ]:
model_name = "training_003"
model_features = pickle.load(open(model_name + ".p", "rb" ))
print(model_features)

In [ ]:
print(Y_train.shape)
Y_train_display = np.array(Y_train)
fig, ax = plt.subplots()
ax.hist(Y_train_display, bins=50)
mean_Y = np.mean(Y_train_display)
ax.axvline(mean_Y, ymin=0, ymax=1, color = 'red', linewidth = 5)
ax.text(7, 10, "mean = {:.3f} s".format(mean_Y), color = "red")
ax.set(xlabel = "TTF (s)", ylabel = "Occurences", title="TTF distribution (train set)")

# *Quelques commentaires sur les modifications:*

**Attention, je fais une distinction "patch" et "batch" !**

Nos données d'entrées sont des séries temporelles. Je les ai découpées en "**patchs**", c'est-à-dire en échantillons de même longueur. 

J'ai choisi cette longueur de sorte que les données d'entraînement soient au même format que les données test (On a des séries test d'une longueur de 150000, j'ai donc découpé en bouts de 150000 les données d'entraînement) : autrement le modèle risque de ne pas être approprié pour réaliser la tâche de prédiction à laquelle on le destine. J'ai gardé l'idée du recouvrement, qui était top.

A partir de là, un échantillon ou point de données correspond à un couple (X, Y) où X est la série temporelle (patch, acoustic_data) et Y le temps avant séisme à la fin du patch (time_to_failure).

On peut ensuite regrouper plusieurs points (X,Y) dans des **"batchs"**. Le regroupement en batch est juste une technique qui sert à fluidifier la descente de gradient: on pourrait en théorie "balancer tout le jeu d'entraînement" au modèle (dans ce cas on dit qu'on fait une "Batch Gradient Descent") mais il est trop gros en pratique et demanderait trop de mémoire. On fait donc une "Mini-Batch Gradient Descent", on "nourrit" le modèle avec des petits paquets de données au fur et à mesure.

A chaque époque, le modèle voit passer l'intégralité du jeu de données.

Cet article résume bien les différentes façons d'aborder la descente de gradient :
https://towardsdatascience.com/batch-mini-batch-stochastic-gradient-descent-7a62ecba642a


**Les données de validation doivent être différentes des données d'entraînement**

J'ai créé un jeu de données de validation à part du jeu d'entraînement.
L'intérêt du jeu de validation est de voir comment le modèle qu'on a entraîné avec les données d'entraînement se démerde avec des données qu'il n'a jamais vues. Au final c'est l'erreur faite sur le jeu de validation qui nous importe: plus cette erreur est faible, plus le modèle fait de bonnes prédictions. 

**En résumé :**

Je charge les données d'entraînement dans X et Y, puis je les découpe avec recouvrement avec la fonction patching() dans X_patch et Y_patch. 
A partir de là, je sépare les données d'entraînement (X_train et Y_train) des données de validation (X_valid et Y_valid).
Au moment d'entraîner le modèle, je répartis aléatoirement X_train et Y_train dans des batchs différents à chaque époque (X_train_batch et Y_train_batch).

**Ce qu'il reste à faire:**
- Essayer d'exécuter le code avec l'intégralité des données d'entraînement et de test. Pour ça, trouver un moyen de charger les données d'entraînement ultra-volumineuses.
- Retrouver le taux d'acquisition du signal pour pouvoir exprimer le time_to_failure avec une unité de temps. 
- Faire un plan d'expérience: il faut essayer plusieurs modèles/architectures, et optimiser les hyperparamètres (learning_rate, nombre de couches cachées) pour chaque modèle. Pour savoir si on est bons, ce serait bien de regarder ce que les gens ont réussi à faire de mieux en termes de prédiction sur kaggle.
- Eventuellement ajouter de l'apprentissage avec Dropout, ajouter de la BatchNorm entre les couches cachées (il me semble que c'est fait automatiquement, mais ça vaut le coup de vérifier)
- Comparer les performances des modèles entre eux avec les données test
- Pour l'instant on a opté pour une méthode de "bourrins" (aucun prétraitement des données) mais cet article https://www.kaggle.com/code/allunia/shaking-earth/notebook#Let's-get-familiar-with-the-data propose des pistes intéressantes (moyennage des séries avec une fenêtre roulante, extraction de paramètres statistiques de la série). 

# *LSTM*

Le LSTM est une architecture adaptée pour les données en séries temporelles.
C'est une version "améliorée" d'un réseau de neurones récurrent (RNN).

Note du site de Pytorch:

"Before getting to the example, note a few things. Pytorch’s LSTM expects all of its inputs to be 3D tensors. The semantics of the axes of these tensors is important. The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input."

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html


In [13]:
### Hyperparameters specific to LSTM

num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr
hidden_size = 1 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers => should stay at 1 unless you want to combine two LSTMs together
N_sub_patches = 250

In [14]:
N_features = 10 #

In [15]:
def data_preparation_2(X_train, Y_train, X_valid, Y_valid, size_patch, N_sub_patches):

    X_train = np.array(th.squeeze(X_train)); Y_train = np.array(Y_train)
    X_valid = np.array(th.squeeze(X_valid)); Y_valid = np.array(Y_valid)

    L_seq = X_train.shape[1]
    L_sub_patch = int(L_seq / N_sub_patches)
    N_features = 10 # mean, std, skew, kurt, min, max, quantiles 0.25, 0.5, 0.75, inter-quartile range
  
    def rearranging_X(X):
        
        X_rearranged = np.zeros((X.shape[0], N_sub_patches, N_features))
        
        for i in range(X.shape[0]):
            for j in range(N_sub_patches):
                xj = X[i,j*L_sub_patch:(j+1)*L_sub_patch]
                X_rearranged[i,j,:] = [xj.mean(), xj.std(), np.double(scipy.stats.skew(xj)), 
                                       np.double(scipy.stats.kurtosis(xj)), xj.min(), xj.max(), 
                                       np.quantile(xj, 0.25), np.quantile(xj, 0.5), np.quantile(xj, 0.75), 
                                       np.quantile(xj, 0.75) - np.quantile(xj, 0.25)]
        return X_rearranged

    X_train_rearranged = rearranging_X(X_train); X_valid_rearranged = rearranging_X(X_valid)

    X_train = th.from_numpy(X_train_rearranged); X_valid = th.from_numpy(X_valid_rearranged)
    Y_train = th.from_numpy(Y_train); Y_valid = th.from_numpy(Y_valid)
    Y_train = th.unsqueeze(Y_train, -1); Y_valid = th.unsqueeze(Y_valid, -1)

    return X_train, Y_train, X_valid, Y_valid

In [ ]:
X_train_, Y_train_, X_valid_, Y_valid_ = data_preparation_2(X_train, Y_train, X_valid, Y_valid, size_patch, N_sub_patches = N_sub_patches)

In [ ]:
#Inspired from https://cnvrg.io/pytorch-lstm/

class LSTM1(nn.Module):
    def __init__(self, N_features, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()

        self.num_layers = num_layers #number of layers
        self.N_features = N_features # Number of features
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(N_features=N_features, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm => Input Shape : (N_batch, L_seq, N_feature)
    
    def forward(self,x):
        
        h_0 = th.zeros(self.num_layers, x.size(0), self.hidden_size) #hidden state
        c_0 = th.zeros(self.num_layers, x.size(0), self.hidden_size) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x.float(), (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = hn
        return out

In [ ]:
#num_classes = 1 #number of output classes => irrelevant here because we are not performing classification
#if user_in == 1:
#    L_seq = X_train_.shape[2]
#else:
#    L_seq = X_train_.shape[1] # length of the sequences

In [ ]:
model = LSTM1(N_features, hidden_size, num_layers, L_seq) #our lstm class

In [ ]:
output = model(th.squeeze(X_train_[0]))

print("output1 = {}; \noutput2 = {}".format(output[0,0], output[1,0]))

In [ ]:
criterion = th.nn.MSELoss()    # mean-squared error for regression
optimizer = th.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
# Problème avec l'entraînement du LSTM => C'est hyper long avec des séries de taille 1500000 !!!
# Plusieurs stratégies:
# - Soit on change le format des données (on fait des coupes de longueur 250-500, préconisées pour les LTSM)
#   risque: grosse perte d'infos, temps trop court pour être informatif. En plus, ne répond pas vraiment au cahier des charges
# - Soit on garde les mêmes séquences, mais on les sous-découpe en patchs sur lesquelles on calcule la variance, la moyenne, la kurtosis.. etc => pleins de paramètres 
#   statistiques et on fabrique une séquence avec autant de paramètres, aussi longue que le nombre de patchs


In [ ]:
# Training LSTM

valid_losses = np.zeros((num_epochs))
train_losses = np.zeros((num_epochs))

min_vl = 1000

best_mvd = []
best_mtd = []

for epoch in range(num_epochs):
    outputs = model.forward(X_train_) #forward pass
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    # obtain the loss function
    loss = criterion(outputs, Y_train_)
 
    loss.backward() #calculates the loss of the loss function

    train_loss = loss.item()
    train_losses[epoch] = train_loss

    with th.no_grad():
        
        valid_output = model(X_valid_)
        valid_loss = criterion(valid_output, Y_valid_)
        valid_losses[epoch] = valid_loss.item()
        valid_differences = valid_output - Y_valid_

        #valid_loss, valid_differences = check_accuracy(X_valid, Y_valid, model)

        if valid_loss < min_vl:

            min_vl = valid_loss
            min_tl = train_loss

            best_mvd = valid_differences

            Y_final = model(X_train_)

            best_mtd = Y_train_ - Y_final
            best_mtd = best_mtd.numpy()
            #best_mtd = best_mtd.cpu().detach().numpy()
        
 
    optimizer.step() #improve from loss, i.e backprop
    if (epoch + 1) % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 5))
fig.subplots_adjust(hspace=0.4, wspace = 0.4)
ax[0,0].plot(train_losses, 'r', label = "training")
ax[0,1].plot(valid_losses, 'b', label = "validation")
ax[1,0].plot(train_losses, 'r', label = "training")
ax[1,0].plot(valid_losses, 'b', label = "validation")
ax[1,0].set(title = "Losses", xlabel = "epochs", ylabel = "MSE")
test = np.array(Y_valid)
ax[1,1].plot(test, 'g')

In [ ]:
model_name = 'LSTM_004'

In [ ]:
plot_and_save_results(train_losses, valid_losses, best_mvd, best_mtd, mm, model_name)

In [ ]:
model_features = pickle.load(open(model_name + "display.p", "rb" ))
print(model_features)